# DiffFusionServer API Example

In this notebook we illustrate the usage of the `DiffFusionServer` REST API via Julia.

Note that `DiffFusionServer` is agnostic to the client application. In this example we demonstrate the use of the `DiffFusionServer` client methods which facilitate easy access to the server.

## Prerequisites

In order to use the API we need to start the DiffFusionServer. 

Run the [StartServer.ipynb](../StartServer.ipynb) notebook from the parent directory to start the server.


Now we are all set.

## Introduction

We use HTTP client methods from the `DiffFusionServer` package to interface with the server. HTTP requests are created under the hood via the `HTTP.jl` package. Data is exchanged in JSON format using the `JSON3.jl` package.

In [ ]:
using DiffFusionServer
using JSON3
using OrderedCollections

As a first test we request the info end point of the API.

In [ ]:
(body, status) = DiffFusionServer.info()
display(status)

If we see a response status of `200` then the server is listening and responding.

Basic element of the API is a flexible object repository. We can query the repository via the *aliases* end point.

In [ ]:
(body, status) = DiffFusionServer.aliases()

We find that there are already some *objects* stored in the repository. These can be used in subsequent object creations.

## Store Objects in Repository

In this step we store a list of prepared objects into the repository. The objects represent curves, models and pricing configurations (*Context*).

The objects are first loaded from file *example_objects.json*.

In [ ]:
data = read("../json/model_objects.json", String)
data = JSON3.read(data)

Now, we can send them to our API.

In [ ]:
for obj in data
    (body, status) = DiffFusionServer.post(obj["alias"], obj)
    println(body)
end

We can verify that the objects are stored either by checking the *repository* variable in the Julia REPL or by querrying the *aliases* end pont again.

In [ ]:
(body, status) = DiffFusionServer.aliases()
for alias in body
    println(alias)
end

## Setting up Cash-flow-based Instruments

Cash-flow-based instruments are stored as lists of cash-flow legs. Each cash-flow leg consists of cash flows. Cash legs are equipped with deterministic notional of FX-resetting notional.

The following example shows a MTM cross currency swap exchanging Euribr6m versus (legacy) USD Libor3m.

Again, we load the swap details from file to keep the example concise.

In [ ]:
data = read("../json/instrument_objects.json", String)
data = JSON3.read(data)

In [ ]:
(body, status) = DiffFusionServer.post("swap/EUR6M-USD3M-jIKbhm", data)
println(body)

## Simulation and Valuation

We have models and instruments stored in the repository. Now, we can run a simulation and calculate scenario prices.

In order to explain the API, we manually compose the simulation call.

In [ ]:
simulation = OrderedDict{String, Any}(
    "typename" => "DiffFusion.Simulation",
    "constructor" => "simple_simulation",
    "model" => "{md/G3}",
    "ch" => "{ch/STD}",
    "times" => [ 0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.0 ],
    "n_paths" => 2^10,
    "kwargs" => OrderedDict{String, Any}(
        "with_progress_bar" => "{true}",
        "brownian_increments" => "{SobolBrownianIncrements}",
    ),
)

The request is structured s follows:

 - The keys *typename* and *constructor* specify the type of object that is created on the server.
 - Text values in braces like *{md/G3}* reference already stored objects in the repository.
 - Dictionary values are passed on to the constructor call on the server.
 - The key *kwargs* specifies additional optional arguments of the constructor. 

In [ ]:
(body, status) = DiffFusionServer.post("sim/G3", simulation)
println(body)

If we wish, we can also inspect the simulation. This may be helpful for debugging.

In [ ]:
(body, status) = DiffFusionServer.get("sim/G3")
display(keys(body))
display(body["times"]')
display(body["X"]["dims"]')

It may also be helpful to check the output in the [StartServer.ipynb](../StartServer.ipynb) notebook and double-check the state of the `repository` variable.

For valuation we need a *path* object. A *path* object links payoffs, simulations and models. This indirection allows a decoupling on models, simulations and instruments.

In [ ]:
path = OrderedDict{String, Any}(
    "typename" => "DiffFusion.Path",
    "constructor" => "path",
    "sim" => "{sim/G3}",
    "ts" => [
        "{yc/USD:SOFR}",
        "{yc/USD:LIB3M}",
        "{yc/EUR:ESTR}",
        "{yc/EUR:XCCY}",
        "{yc/EUR:EURIBOR6M}",
        "{yc/GBP:SONIA}",
        "{yc/GBP:XCCY}",
        "{pa/USD:SOFR}",
        "{pa/USD:LIB3M}",
        "{pa/EUR:ESTR}",
        "{pa/EUR:EURIBOR6M}",
        "{pa/GBP:SONIA}",
        "{pa/EUR-USD}",
        "{pa/GBP-USD}",
        "{pa/EUHICP}",
        "{pa/NIK-FUT}",
    ],
    "ctx" => "{ct/STD}",
    "ip" => "{LinearPathInterpolation}",
)

We see that the *path* arguments are all references to objects that are already stored in the repository.

In [ ]:
(body, status) = DiffFusionServer.post("path/G3", path)
println(body)

Scenario pricing is implemented as a function in *DiffFusion*. All inputs to the function are stored in the repository and can be used. 

In [ ]:
cube = OrderedDict{String, Any}(
    "typename" => "DiffFusion.ScenarioCube",
    "constructor" => "scenarios",
    "legs" => "{swap/EUR6M-USD3M-jIKbhm}",
    "times" => [ 0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.0 ],
    "path" => "{path/G3}",
    "discount_curve_key" => "nothing",
)

In [ ]:
(body, status) = DiffFusionServer.post("cube/EUR6M-USD3M-jIKbhm", cube)
println(body)

Again, we can query the result and inspect the details.

In [ ]:
(body, status) = DiffFusionServer.get("cube/EUR6M-USD3M-jIKbhm")
display(keys(body))
display(body["times"]')
display(body["X"]["dims"]')

Finally, we calculate the expected positive exposure (EE) for our example swap.

In [ ]:
cube = OrderedDict{String, Any}(
    "typename" => "DiffFusion.ScenarioCube",
    "constructor" => "expected_exposure",
    "scens" => "{cube/EUR6M-USD3M-jIKbhm}",
    "gross_leg" => "{false}",
    "average_paths" => "{true}",
    "aggregate_legs" => "{true}",
)

In [ ]:
(body, status) = DiffFusionServer.post("cube/EUR6M-USD3M-jIKbhm/EE", cube)
println(body)

And we get the result as before.

In [ ]:
(body, status) = DiffFusionServer.get("cube/EUR6M-USD3M-jIKbhm/EE")
display(keys(body))
display(body["times"]')
display(body["X"]["dims"]')

Our swap EE is finally:

In [ ]:
display(body["times"]')
display(body["X"]["data"]')

As a cross-check we can also calculate expected future market values.

In [ ]:
cube = OrderedDict{String, Any}(
    "typename" => "DiffFusion.ScenarioCube",
    "constructor" => "aggregate",
    "scens" => "{cube/EUR6M-USD3M-jIKbhm}",
    "average_paths" => "{true}",
    "aggregate_legs" => "{true}",
)
(body, status) = DiffFusionServer.post("cube/EUR6M-USD3M-jIKbhm/MV", cube)
println(body)
(body, status) = DiffFusionServer.get("cube/EUR6M-USD3M-jIKbhm/MV")
display(body["times"]')
display(body["X"]["data"]')

We see that the market value is negative for *t=0*. Consequently, the EE is zero at the initial simulation time point.